In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd 
import sys, os
from src import *
from helpers import *

### KERNEL CONFIGURATION
Set up an application to be accelerated by loading the different kernel configurations into the kernel memory

In [2]:
# Load an existing kernel into memory
kmem_pos = 1
kmem_word = 0x802b

kmem = KER_CONF()
kmem.set_word(kmem_word, kmem_pos)
kmem.get_kernel_info(kmem_pos)

This kernel uses 44 instruction words starting at IMEM address 0.
It uses column(s): 0.
The SRF is located in SPM bank 0.


In [3]:
# Create a new kernel
kmem_pos = 2

# Kernel configuration parameters
num_instructions=1
imem_add_start=0
column_usage=1
srf_spm_addres=0

kmem.set_params(num_instructions, imem_add_start, column_usage, srf_spm_addres, kmem_pos)
print("Hex representation: " + kmem.get_word_in_hex(kmem_pos))
kmem.get_kernel_info(kmem_pos)

Hex representation: 0x8000
This kernel uses 1 instruction words starting at IMEM address 0.
It uses column(s): 0.
The SRF is located in SPM bank 0.


### Loop Control Unit IMEM 

In [4]:
# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0x1c00

lcu_imem = LCU_IMEM()
lcu_imem.set_word(imem_word, imem_pos)
lcu_imem.get_instruction_info(imem_pos)

Immediate value: 0
LCU is in loop control mode
Exiting out of kernel
No LCU registers are being written


In [5]:
# Create a new instruction
imem_pos = 2

# Define instruction parameters
imm=3
rf_wsel=2
rf_we=1
alu_op=LCU_ALU_OPS.SADD
br_mode=0
muxb_sel=LCU_MUXB_SEL.ONE
muxa_sel=LCU_MUXA_SEL.ZERO

lcu_imem.set_params(imm, rf_wsel, rf_we, alu_op, br_mode, muxb_sel, muxa_sel, imem_pos)
print("Hex representation: " + lcu_imem.get_word_in_hex(imem_pos))
lcu_imem.get_instruction_info(imem_pos)

Hex representation: 0xdc383
Immediate value: 3
LCU is in loop control mode
Performing ALU operation SADD between operands ZERO and ONE
Writing ALU result to LCU register 2


### Load Store Unit IMEM 

In [6]:
# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0x8b49f

lsu_imem = LSU_IMEM()
lsu_imem.set_word(imem_word, imem_pos)
lsu_imem.get_instruction_info(imem_pos)

Performing STORE to SPM from VWR_B
Performing ALU operation LOR between operands R6 and ZERO
Writing ALU result to LSU register 7


In [7]:
# Create a new instruction
imem_pos = 2

# Define instruction parameters
rf_wsel=2
rf_we=1
alu_op=LSU_ALU_OPS.SRL
muxb_sel=LSU_MUXA_SEL.R5
muxa_sel=LSU_MUXB_SEL.TWO
vwr_shuf_op=SHUFFLE_SEL.CONCAT_BITREV_UPPER
vwr_shuf_sel=LSU_OP_MODE.SHUFFLE

lsu_imem.set_params(rf_wsel, rf_we, alu_op, muxb_sel, muxa_sel, vwr_shuf_op, vwr_shuf_sel, imem_pos)
print("Hex representation: " + lsu_imem.get_word_in_hex(imem_pos))
lsu_imem.get_instruction_info(imem_pos)

Hex representation: 0xe5aea
Shuffling VWR A and B data into VWR C using operation CONCAT_BITREV_UPPER
Performing ALU operation SRL between operands TWO and R5
Writing ALU result to LSU register 2


### Multiplexer Control Unit IMEM

In [8]:
# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0x180

mxcu_imem = MXCU_IMEM()
mxcu_imem.set_word(imem_word, imem_pos)
mxcu_imem.get_instruction_info(imem_pos)

Not writing to VWRs
Reading from SRF index 6
Performing ALU operation NOP between operands R0 and R0
No MXCU registers are being written


In [9]:
# Create a new instruction
imem_pos = 2

# Define instruction parameters
vwr_row_we = [1, 1, 1, 0]
vwr_sel = MXCU_VWR_SEL.VWR_B.value
srf_sel = 3
alu_srf_write = ALU_SRF_WRITE.MXCU
srf_we = 1
rf_wsel = 0 
rf_we = 0 
alu_op =  MXCU_ALU_OPS.SADD
muxb_sel = MXCU_MUXB_SEL.R0
muxa_sel = MXCU_MUXA_SEL.TWO

mxcu_imem.set_params(vwr_row_we, vwr_sel, srf_sel, alu_srf_write, srf_we, rf_wsel, rf_we, alu_op, muxb_sel, muxa_sel, imem_pos)
mxcu_imem.get_instruction_info(imem_pos)

Writing to VWR rows [1 2 3] of VWR_B
Writing from MXCU ALU to SRF register 3
Performing ALU operation SADD between operands TWO and R0
No MXCU registers are being written


### Reconfigurable Cell IMEM

In [10]:
# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0xe923

rc_imem = RC_IMEM()
rc_imem.set_word(imem_word, imem_pos)
rc_imem.get_instruction_info(imem_pos)

Performing ALU operation LOR between operands SRF and ZERO
ALU is performing operations with 32-bit precision
Writing ALU result to RC register 1


In [11]:
# Create a new instruction
imem_pos = 2

# Define instruction parameters
rf_wsel = 1 
rf_we = 1 
muxf_sel = RC_MUXF_SEL.RCT 
alu_op =  RC_ALU_OPS.INB_SF_INA
op_mode = 0 #Always keep this to zero; 16-bit mode is not supported yet
muxb_sel =  RC_MUXA_SEL.VWR_A
muxa_sel = RC_MUXA_SEL.VWR_B

rc_imem.set_params(rf_wsel, rf_we, muxf_sel, alu_op, op_mode, muxb_sel, muxa_sel, imem_pos)
rc_imem.get_instruction_info(imem_pos)

Output VWR_B if sign flag of RCT == 1, else output VWR_A
Writing ALU result to RC register 1


## Putting it all together: Instruction memory

### Process existing kernel
Load an existing kernel (in the form of an excel sheet where each row is a clock cycle and each column is a specialized slot) and use the simulator to understand what is going on in each element at a given clock cycle

In [12]:
kernel_path = "kernels/mf_q64_erosion/"
df = pd.read_csv(kernel_path + "instructions.csv")
print("The instruction memory has {0} entries.".format(len(df)))
df.head()

The instruction memory has 512 entries.


,LCU,LSU,MXCU,RC0,RC1,RC2,RC3,KMEM
0,0x0,0x5c49f,0x0,0x0,0x0,0x0,0x0,0x802b
1,0x9c500,0x43d3f,0x180,0x0,0x0,0x0,0x0,0x0
2,0x98fc0,0x4bd3f,0x40,0x0,0x0,0x0,0x0,0x0
3,0xf8f43,0x53c98,0x0,0x0,0x0,0x0,0x0,0x0
4,0xb8f80,0x539,0x4ce9000,0x0,0x0,0x0,0x0,0x0


In [13]:
# Set an IMEM object and load kernel 0
imem = IMEM(df)
imem.load_kernel(0)

In [14]:
# Make sure that the last kernel value is the exit instruction
n_instr, imem_add, _, _ = imem.kmem.get_params(0)
lcu_instr = imem.lcu_imem.get_instruction_info(n_instr + imem_add)

Immediate value: 0
LCU is in loop control mode
Exiting out of kernel
No LCU registers are being written


In [15]:
# Print what's going on at a given imem position
imem.get_pos_summary(imem_pos=0)

****RC0****
No ALU operation
No RC registers are being written
****RC1****
No ALU operation
No RC registers are being written
****RC2****
No ALU operation
No RC registers are being written
****RC3****
No ALU operation
No RC registers are being written
****LSU****
Performing LOAD from SPM to SRF
Performing ALU operation LOR between operands SRF and ZERO
Writing ALU result to LSU register 7
****LCU****
Immediate value: 0
LCU is in loop control mode
No LCU ALU Operation is performed
No LCU registers are being written
****MXCU****
Not writing to VWRs
Reading from SRF index 0
Performing ALU operation NOP between operands R0 and R0
No MXCU registers are being written


### Create a new kernel
Write your own kernel by populating the instruction memory of each specialized slot one at a time. In this example, we develop a  very simple kernel whose only instruction is an exit. The resulting ISAs of each element are loaded into a pandas dataframe and saved to a CSV and C header file, which can then be copied into a VWR2A testbench.

In [16]:
# Set up kernel
imem = IMEM()

num_instructions = 1
imem_add_start = 0
column_usage = 1
srf_spm_address = 0
pos = 0

imem.kmem.set_params(num_instructions, imem_add_start, column_usage, srf_spm_address, pos)
imem.kmem.get_kernel_info(pos)

This kernel uses 1 instruction words starting at IMEM address 0.
It uses column(s): 0.
The SRF is located in SPM bank 0.


In [17]:
# Write LCU instruction
imem.lcu_imem.set_params(alu_op=LCU_ALU_OPS.EXIT, pos=0)

In [18]:
# Get instruction dataframe
df_out = imem.get_df()
df_out.head()

,LCU,LSU,MXCU,RC0,RC1,RC2,RC3,KMEM
0,0x1c00,0x4c80,0x0,0x0,0x0,0x0,0x0,0x8000
1,0x0,0x4c80,0x0,0x0,0x0,0x0,0x0,0x0
2,0x0,0x4c80,0x0,0x0,0x0,0x0,0x0,0x0
3,0x0,0x4c80,0x0,0x0,0x0,0x0,0x0,0x0
4,0x0,0x4c80,0x0,0x0,0x0,0x0,0x0,0x0


In [19]:
# Save to file
kernel_path = 'kernels/exit/'
# Save CSV of instructions to be easily re-loaded and fixed
df_out.to_csv(kernel_path + 'instructions.csv')
# Save header file used to load the instructions into a VWR2A RTL testbench for a functional simulation
dataframe_to_header_file(df_out, kernel_path)

### A more complicated example
Now that we know how to populate the IMEMs of the specialized slots, we will write a more complicated kernel that adds two vectors together.

In [20]:
imem = IMEM()

In [21]:
# Instruction 0: Load SPM address zero into SRF. The last SRF value is the number of iterations per slice. 
pos=0

rf_wsel=7
rf_we=1
alu_op=LSU_ALU_OPS.LOR
muxb_sel=LSU_MUXA_SEL.ZERO
muxa_sel=LSU_MUXB_SEL.SRF
vwr_shuf_op=LSU_VWR_SEL.SRF
vwr_shuf_sel=LSU_OP_MODE.LOAD

imem.lsu_imem.set_params(rf_wsel, rf_we, alu_op, muxb_sel, muxa_sel, vwr_shuf_op, vwr_shuf_sel, pos)
imem.lsu_imem.get_instruction_info(pos)

Performing LOAD from SPM to SRF
Performing ALU operation LOR between operands SRF and ZERO
Writing ALU result to LSU register 7
